In [1]:
from keras.layers import Input, Dropout, Dense, Flatten, concatenate
from keras.layers.convolutional import MaxPooling3D, Conv3D, Conv3DTranspose
from keras.models import Model

Using TensorFlow backend.


In [2]:
_input = Input(shape=(1, 3, 9600, 3600))

In [3]:
conv1 = Conv3D(32, (1, 2, 2), strides=(1, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first',
                   name='block1_conv1')(_input)
pool1 = MaxPooling3D((1, 2, 2), strides=(1, 2, 2),
                         data_format='channels_first',
                         name='block1_pool')(conv1)

# Block 2
conv2 = Conv3D(64, (1, 2, 2), strides=(1, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first',
                   name='block2_conv1')(pool1)
pool2 = MaxPooling3D((1, 2, 2), strides=(1, 2, 2),
                         data_format='channels_first',
                         name='block2_pool')(conv2)

# Block 3
conv3 = Conv3D(128, (3, 2, 2), strides=(3, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first',
                   name='block3_conv1')(pool2)
pool3 = MaxPooling3D((1, 2, 2), strides=(1, 2, 2),
                         data_format='channels_first',
                         name='block3_pool')(conv3)

# Block 4
conv4 = Conv3D(256, (1, 2, 2), strides=(1, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first',
                   name='block4_conv1')(pool3)
pool4 = MaxPooling3D((1, 2, 2), strides=(1, 2, 2), name='block4_pool',
                         data_format='channels_first')(conv4)

# Block 5
conv5 = Conv3D(512, (1, 2, 2), strides=(1, 2, 2), activation='relu',
                   padding='same', data_format='channels_first')(pool4)

# Block 6^T
up6 = concatenate([Conv3DTranspose(256, (1, 4, 4),
                                       strides=(1, 4, 4), padding='same',
                                       data_format='channels_first')(conv5),
                       conv4], axis=1)
conv6 = Conv3D(256, (1, 2, 2), strides=(1, 2, 2), activation='relu',
                   padding='same', data_format='channels_first')(up6)

# Block 7^T
up7 = concatenate([Conv3DTranspose(128, (1, 4, 4),
                                       strides=(1, 4, 4), padding='same',
                                       data_format='channels_first')(conv6),
                       conv3], axis=1)
conv7 = Conv3D(128, (1, 2, 2), strides=(1, 2, 2), activation='relu', 
                   padding='same', data_format='channels_first')(up7)

# Block 8^T
up8 = concatenate([Conv3DTranspose(64, (3, 4, 4),
                                       strides=(3, 4, 7), padding='same',
                                       data_format='channels_first')(conv7),
                       conv2], axis=1)
conv8 = Conv3D(64, (1, 3, 6), activation='relu', padding='same', 
                   data_format='channels_first')(up8)

# Block 9^T
up9 = concatenate([Conv3DTranspose(32, (3, 3, 6),
                                       strides=(3, 3, 6), padding='same',
                                       data_format='channels_first')(up8),
                       conv1], axis=1)
conv9 = Conv3D(32, (1, 3, 6), activation='relu', padding='same', 
                   data_format='channels_first')(up9)

model = Model(_input, conv9)
model.compile(loss='categorical_crossentropy', optimizer='sgd',
                 metrics=['accuracy'])

ValueError: `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 256, 1, 76, 28), (None, 256, 1, 75, 28)]